In [ ]:
import shap
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

# Load data for two time windows
df_t1 = pd.read_csv('data_time1.csv')
df_t2 = pd.read_csv('data_time2.csv')

# Train a model (on T1 or full)
model = XGBClassifier().fit(df_t1.drop('target', axis=1), df_t1['target'])

# SHAP analysis
explainer = shap.TreeExplainer(model)
shap_values_t1 = explainer.shap_values(df_t1.drop('target', axis=1))
shap_values_t2 = explainer.shap_values(df_t2.drop('target', axis=1))

# Aggregate
mean_shap_t1 = np.abs(shap_values_t1).mean(axis=0)
mean_shap_t2 = np.abs(shap_values_t2).mean(axis=0)

# Compare
df_compare = pd.DataFrame({
    'feature': df_t1.drop('target', axis=1).columns,
    'SHAP_t1': mean_shap_t1,
    'SHAP_t2': mean_shap_t2,
    'drift': mean_shap_t2 - mean_shap_t1
})

df_compare.sort_values(by='drift', key=abs, ascending=False).head(10).plot(
    x='feature', y=['SHAP_t1', 'SHAP_t2'], kind='bar'
)
plt.title("SHAP Feature Drift Over Time")
plt.show()